In [1]:
#load libraries
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
import requests
import csv
import numpy as np
import pandas as pd
print("Libraries Imported.")

Libraries Imported.


In [2]:
#PART 1 of Project
#parse data from wikipedia with Beautiful Soup and Python
req = requests.get("https://en.wikipedia.org/w/index.php?title=List_of_postal_codes_of_Canada:_M&oldid=890001695")
soup = BeautifulSoup(req.content,'html.parser')
table = soup.find_all('table')

#create dataframe from parsed data
df = pd.read_html(str(table))[0]
df

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
...,...,...,...
283,M8Z,Etobicoke,Mimico NW
284,M8Z,Etobicoke,The Queensway West
285,M8Z,Etobicoke,Royal York South West
286,M8Z,Etobicoke,South of Bloor


In [3]:
#remove all 'not assigned' boroughs
z=df.loc[(df['Borough'] != "Not assigned")] 
print(z)

    Postcode           Borough             Neighbourhood
2        M3A        North York                 Parkwoods
3        M4A        North York          Victoria Village
4        M5A  Downtown Toronto              Harbourfront
5        M5A  Downtown Toronto               Regent Park
6        M6A        North York          Lawrence Heights
..       ...               ...                       ...
282      M8Z         Etobicoke  Kingsway Park South West
283      M8Z         Etobicoke                 Mimico NW
284      M8Z         Etobicoke        The Queensway West
285      M8Z         Etobicoke     Royal York South West
286      M8Z         Etobicoke            South of Bloor

[211 rows x 3 columns]


In [4]:
#group by Postal Code and Borough.  Use lambda to add a comma to the Boroughs
grouped=z.groupby(['Postcode','Borough'], as_index=False).agg(lambda x: ', '.join(x))
grouped

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [5]:
#Replace Not assigned data
grouped['Neighbourhood']=grouped['Neighbourhood'].replace('Not assigned', grouped['Borough'])
grouped.style.hide_index()

Postcode,Borough,Neighbourhood
M1B,Scarborough,"Rouge, Malvern"
M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
M1E,Scarborough,"Guildwood, Morningside, West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae
M1J,Scarborough,Scarborough Village
M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
M1N,Scarborough,"Birch Cliff, Cliffside West"


In [6]:
#Get shape of dataframe
grouped.shape 

(103, 3)

In [7]:
#PART 2 of Project
#Get the latitude and the longitude coordinates of each neighborhood.
#Failed to load.  Read CSV to get coords.

!pip install geocoder
import geocoder
geo=pd.read_csv('Geospatial_Coordinates.csv')
geo

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476
...,...,...,...
98,M9N,43.706876,-79.518188
99,M9P,43.696319,-79.532242
100,M9R,43.688905,-79.554724
101,M9V,43.739416,-79.588437


In [8]:
#Renamed grouped dataframe to make merger of frames easier
grouped.rename(columns={"Postcode" : "Postal Code"}, inplace=True)
grouped

,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


In [9]:
#Merged frames 'grouped' and 'geo'
merged_grouped = pd.merge(grouped, geo, on='Postal Code')
merged_grouped.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [10]:
#Import required libraries for mapping, plotting and clustering

import folium

import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json 

!pip install geopy
from geopy.geocoders import Nominatim

import requests #
from pandas.io.json import json_normalize 

import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans

print('Libraries imported.')

Libraries imported.


In [11]:
#Link to Foursquare
CLIENT_ID = 'FOONXVWYCI4P53VX4WN5H3GWAMFB5JIZBD0KLUI1AI0UFCLL' # your Foursquare ID
CLIENT_SECRET = 'OTN4Y4O2HQVT3OZSR3W5CV3QKFFY00OCPEDKSW1WHLWUWHTJ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 100
RADIUS = 500
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: FOONXVWYCI4P53VX4WN5H3GWAMFB5JIZBD0KLUI1AI0UFCLL
CLIENT_SECRET:OTN4Y4O2HQVT3OZSR3W5CV3QKFFY00OCPEDKSW1WHLWUWHTJ


In [12]:
#Find Toronto using 
address = "Toronto, ON"

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


In [13]:
# create map of Toronto using Folium
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)
map_toronto

In [14]:
#Frame for data
merged_grouped.head(20)

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park",43.727929,-79.262029
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


In [15]:
#Create Toronto Map with Markers

for lat, lng, borough, neighborhood in zip(
        merged_grouped['Latitude'], 
        merged_grouped['Longitude'], 
        merged_grouped['Borough'], 
        merged_grouped['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [16]:
#Explore boroughs that contain the word Toronto ONLY
toronto_table=merged_grouped[merged_grouped['Borough'].str.contains("Toronto")].reset_index(drop=True)
toronto_table

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [17]:
#Update Map for 4 Neighbourhoods ONLY
for lat, lng, borough, neighborhood in zip(
        toronto_table['Latitude'], 
        toronto_table['Longitude'], 
        toronto_table['Borough'], 
        toronto_table['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.6,
        parse_html=False).add_to(map_toronto)  

map_toronto

In [18]:
#Find the 1st Neighbourhood in Toronto
main_neighbourhood = toronto_table.loc[0, 'Neighbourhood']
main_neighbourhood

'The Beaches'

In [20]:
#Use Foursquare API to search The Beaches and get JSON file
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    43.68, 
   -79.29, 
    RADIUS, 
    LIMIT)
results = requests.get(url).json()

In [21]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [22]:
#Normailize the file and create a new dataframe
venues = results['response']['groups'][0]['items']
nearby_venues = pd.json_normalize(venues)
nearby_venues


# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]
nearby_venues.head()

,name,categories,lat,lng
0,The Beech Tree,Gastropub,43.680493,-79.288846
1,Beaches Bake Shop,Bakery,43.680363,-79.289692
2,The Feathers Pub,Pub,43.680501,-79.287522
3,Glen Manor Ravine,Trail,43.676821,-79.293942
4,The Real Jerk Beaches,Caribbean Restaurant,43.680781,-79.285727


In [23]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

17 venues were returned by Foursquare.


In [33]:
#Create a Venues Map
venues_map = folium.Map(location=[43.68,-79.29], zoom_start=15)

# add a blue circle marker to represent The Beaches
folium.features.CircleMarker(
    [43.68, -79.29],
    radius=10,
    color='blue',
    popup='The Beaches',
    fill = True,
    fill_color = 'blue',
    fill_opacity = 0.6
).add_to(venues_map)

# add all venues as red circle markers
for lat, lng, label in zip(nearby_venues.lat, nearby_venues.lng, nearby_venues.categories):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='red',
        popup=label,
        fill = True,
        fill_color='red',
        fill_opacity=0.6
    ).add_to(venues_map)
venues_map